## Chapter 2: Motif Count /Profile

In [12]:
import random

In [1]:
def approx_pattern_matching(pattern: str, text: str, d: int) -> list[int]: #1.8.2
    ind = []
    for i in range(0,len(text)-len(pattern)+1):
        kmer = text[i:i+len(pattern)]
        #if text=="GAAAATT":
            #print(kmer)
        if hamming_distance(kmer,pattern)<=d:
            ind.append(i)
    return ind
def hamming_distance(p: str, q: str) -> int: #1.8.1
    if len(p)!=len(q):
        return -1
    ham_count = 0
    for i in range(0,len(p)):
        if p[i]!=q[i]:
            ham_count+=1
    
    return ham_count
def neighbors(pattern: str,d: int) -> list[str]:
    if d==0:
        return [pattern]
    if len(pattern)==1:
        return ["A","C","G","T"]
    Neighborhood = []
    suffixNeighborhood = neighbors(pattern[1:],d)
    for text in suffixNeighborhood:
        if hamming_distance(pattern[1:],text)<d:
            for nucleotide in ["A","T","G","C"]:
                Neighborhood.append(nucleotide+text)
        else:
            Neighborhood.append(pattern[0]+text)
    return Neighborhood

### Motif Enumeration 
Finding motif with mismatches \
Motif = pattern that appears in all dna array at least once with at most d mismatches\

Dna = [dna1,dna2,dna3...,dna_n]\


MotifEnumeration(Dna, k, d)
    Patterns ← an empty set
    for each k-mer Pattern in the first string in Dna
        for each k-mer Pattern’ differing from Pattern by at most d mismatches
            if Pattern' appears in each string from Dna with at most d mismatches
                add Pattern' to Patterns
    remove duplicates from Patterns
    return Patterns

In [2]:
def motif_enumeration(dna: list[str], k: int, d: int) -> list[str]:
    patterns = []
    for i in range(0,len(dna[0])-k+1):
        Pattern = dna[0][i:i+k]
        neighborhood = neighbors(Pattern,d)
        for neighbor in neighborhood:
            for array in dna:
                in_all= [len(approx_pattern_matching(neighbor, array, d))>0 for array in dna]
                #print(neighbor,in_all)
                if all(in_all) and neighbor not in patterns:
                    patterns.append(neighbor)
    return patterns


### Motif Finding Problem: Profile Matrices and Consensus Strings
Position is conserved if all nucleotides at pos in dna arrays are the same\
Score(motifs) = sum(#unpopular nucleotides at pos_i) for i=0:len(motif)\
\
Min possible score = 0 (if all are conserved) -> we want to minimise score to find the consensus motif\
max possible score = motif_len * ceil(#dna_array*(3/4)) \
a nucleotide needs around 3/4th presence to be conserved





Count(motifs) = count of each nuc at each pos over all motifs\
\
Profile(motifs) = ratio of each nuc at each pos (count/#array)\
\
Consensus(motifs) = choosing nuc at each pos with highest profile score, (choosing most conserved nuc at each pos)

Brute Force Algo -> compute score of every kmer combinations that will minimise score(motifs)\

Calculate hamming distance of between kmer and consensus(motif):\
Score(motifs) = hamming_dist(Consensus(Motifs),Motifs)

#### Median String problem
Median string is a kmer pattern minimising d(kmer,Dna) over all possible kmers


MedianString(Dna, k)
    distance ← ∞
    for each k-mer Pattern from AA…AA to TT…TT
        if distance > d(Pattern, Dna)
            distance ← d(Pattern, Dna)
            Median ← Pattern
    return Median

In [5]:
def distance_between_pattern_and_strings(pattern: str, dna: list[str]) -> int: #finding min hamming distance in every array, and then adding all min_ham_dist for all dna strings (this gives us min score)
    k = len(pattern)
    distance = 0
    for array in dna:
        hamming_d = float('inf')
        for i in range(0,len(array)-k+1):
            kmer = array[i:i+k]
            curr_ham = hamming_distance(kmer,pattern)
            if hamming_d > curr_ham:
                hamming_d = curr_ham
        distance = distance + hamming_d
    return distance
def median_string(dna: list[str],k: int) -> str: #knowing min scores for all possible patterns we can find kmer that gives min score
    distance = float('inf')
    patterns = all_k_strings(k) #finding all strings of len=k
    for pattern in patterns:
        curr_d  = distance_between_pattern_and_strings(pattern, dna)
        if distance > curr_d:
            distance = curr_d
            median=pattern
    return median
def all_k_strings(k: int) -> list[str]:
    start = "".join(['A' for _ in range(k)])
    return neighbors(start,k)


### Profile Most Probable kmer



In [ ]:
def profile_most_probable_kmer(text: str, k: int,
                               profile: list[dict[str, float]]) -> str:
    """Identifies the most probable k-mer according to a given profile matrix.

    The profile matrix is represented as a list of columns, where the i-th element is a map
    whose keys are strings ("A", "C", "G", and "T") and whose values represent the probability
    associated with this symbol in the i-th column of the profile matrix.
    """
    max_prob = 0
    max_motif = ''
    for i in range(0,len(text)-k+1): #all kmers in text
        kmer = text[i:i+k]
        prob = 1
        for ind in range(0,k): #each nuc in kmer
            nuc = kmer[ind]
            prob = prob * profile[ind][nuc] #prob for nuc in profile
        if max_prob < prob: #finding kmer with max prob
            max_prob = prob
            max_motif = kmer
    return max_motif #return kmer with max prob

In [6]:
def profile_least_probable_kmer(text:str,k:int,profile:list[dict[str,float]])->str:
    least_prob = -1
    least_motif = ''
    for i in range(0,len(text)-k+1): #all kmers in text
        kmer = text[i:i+k]
        prob = 1
        for ind in range(0,k): #each nuc in kmer
            nuc = kmer[ind]
            prob = prob * profile[ind][nuc] #prob for nuc in profile
        if least_prob > prob: #finding kmer with max prob
            least_prob = prob
            least_motif = kmer
    return least_motif #return kmer with max prob

### Randomised Motif Search
To improve brute force motif search


RandomizedMotifSearch(Dna, k, t)
    randomly select k-mers Motifs = (Motif1, …, Motift) in each string from Dna
    BestMotifs ← Motifs
    while forever
        Profile ← Profile(Motifs)
        Motifs ← Motifs(Profile, Dna)
        if Score(Motifs) < Score(BestMotifs)
            BestMotifs ← Motifs
        else
            return BestMotifs

In [14]:
def randomized_motif_search(dna: list[str], k: int, t: int)-> list[str]:
    best_motifs = randomized_motif_search_single(dna,k,t)
    curr_score = score(best_motifs)
    for i in range(0,999):
        print(i)
        motifs = randomized_motif_search_single(dna,k,t)
        if score(motifs) < curr_score:
            best_motifs = motifs
            curr_score = score(motifs)
    return best_motifs
def randomized_motif_search_single(dna: list[str], k: int, t: int)-> list[str]:
    rand_ind = []
    for array in dna:
        rand_ind.append(random.randint(0,len(array)-k))
    motifs = [dna[i][rand_ind[i]:rand_ind[i]+k] for i in range(len(dna))]
    
    print("rand", motifs)
    best_motifs = motifs
    curr_score = score(best_motifs)
    while True:
        Profile = profile(motifs)
        print(Profile)
        motifs = most_probable_motifs(dna,k,Profile)
        print("motifs:", motifs)
        if score(motifs) < curr_score:
            best_motifs = motifs
            curr_score = score(motifs)
        else:
            return best_motifs
        
def profile(motifs: list[str])-> list[dict[str,float]]:
    profile_matrix = []
    total = len(motifs)+4
    for i in range(len(motifs[0])):
        motif_dict = {}
        motif_dict['A'] = (sum([motif[i]=='A' for motif in motifs])+1)/total
        motif_dict['T'] = (sum([motif[i]=='T' for motif in motifs])+1)/total
        motif_dict['G'] = (sum([motif[i]=='G' for motif in motifs])+1)/total
        motif_dict['C'] = (sum([motif[i]=='C' for motif in motifs])+1)/total
        profile_matrix.append(motif_dict)

    return profile_matrix
def most_probable_motifs(dna: list[str],  k: int, profile: list[dict[str, float]])-> str:
    motif_set = []
    for array in dna:
        motif_set.append(profile_most_probable_kmer(array,k,profile))
    return motif_set
def profile_most_probable_kmer(text: str, k: int, profile: list[dict[str, float]]) -> str:
    max_prob = 0
    max_motif = ''
    for i in range(0,len(text)-k+1):
        kmer = text[i:i+k]
        prob = 1
        for ind in range(0,k):
            nuc = kmer[ind]
            prob = prob * profile[ind][nuc]
        if max_prob < prob:
            max_prob = prob
            max_motif = kmer
    print('max_motif',max_motif)
    return max_motif
            
def consensus(motifs):
    """Find the consensus string of the given motifs."""
    consensus = ""
    for i in range(len(motifs[0])):
        column = [motif[i] for motif in motifs]
        consensus += max(set(column), key=column.count)
    return consensus
#max possible score is (3/4*t*k)
def score(motifs):
    """Calculate the score of the given motifs against the consensus string."""
    cons = consensus(motifs)
    score = 0
    for motif in motifs:
        for i in range(len(motif)):
            if motif[i] != cons[i]:
                score += 1
    return score

### Gibbs Motif Search


In [16]:
def gibbs_sampler(dna: list[str], k: int, t: int, n: int) -> list[str]:
    """Implements the GibbsSampling algorithm for motif finding."""
    best_motifs = gibbs_sampler_single(dna,k,t,n)
    curr_score = score(best_motifs)
    for i in range(0,999):
        motifs = gibbs_sampler_single(dna,k,t,n)
        if score(motifs) < curr_score:
            best_motifs = motifs
            curr_score = score(motifs)
    return best_motifs
def gibbs_sampler_single(dna: list[str], k: int, t: int, n: int) -> list[str]:
    rand_ind = []
    for array in dna:
        rand_ind.append(random.randint(0,len(array)-k))
    motifs = [dna[i][rand_ind[i]:rand_ind[i]+k] for i in range(len(dna))]
    best_motifs = motifs
    for j in range(1,n+1):
        i = random.randint(0,t-1) #choose random dna array to remove
        Profile = profile(motifs[:i]+motifs[i+1:]) #build profile out of remaining t-1
        motif_i = profile_rand_kmer(dna[i],k,Profile) #
        motifs = best_motifs[:i]+[motif_i]+best_motifs[i+1:]
        if score(motifs) < score(best_motifs):
            best_motifs = motifs
    return best_motifs
def profile_rand_kmer(text: str, k:int,profile: list[dict[str, float]])-> str: #select a k-mer from a given text based on a profile matrix
    kmer_pr = []
    for i in range(0,len(text)-k+1):
        kmer = text[i:i+k]
        pr = 1
        for ind in range(0,len(kmer)):
            pr = pr* profile[ind][kmer[ind]]
        kmer_pr.append(pr)
    final_pr = [kmer_pr[j]/sum(kmer_pr) for j in range(0,len(kmer_pr))]
    chosen_ind = random.choices(range(len(text)-k+1), weights=final_pr, k=1)[0]
    return text[chosen_ind:chosen_ind+k]
def consensus(motifs):
    """Find the consensus string of the given motifs."""
    consensus = ""
    for i in range(len(motifs[0])):
        column = [motif[i] for motif in motifs]
        consensus += max(set(column), key=column.count)
    return consensus

def score(motifs):
    """Calculate the score of the given motifs against the consensus string."""
    cons = consensus(motifs)
    score = 0
    for motif in motifs:
        for i in range(len(motif)):
            if motif[i] != cons[i]:
                score += 1
    return score
def profile(motifs: list[str])-> list[dict[str,float]]:
    profile_matrix = []
    total = len(motifs)+4
    for i in range(len(motifs[0])):
        motif_dict = {}
        motif_dict['A'] = (sum([motif[i]=='A' for motif in motifs])+1)/total
        motif_dict['T'] = (sum([motif[i]=='T' for motif in motifs])+1)/total
        motif_dict['G'] = (sum([motif[i]=='G' for motif in motifs])+1)/total
        motif_dict['C'] = (sum([motif[i]=='C' for motif in motifs])+1)/total
        profile_matrix.append(motif_dict)

    return profile_matrix

In [17]:
dna = ['CGCCCCTCTCGGGGGTGTTCAGTAAACGGCCA','GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG','TAGTACCGAGACCGAAAGAAGTATACAGGCGT','TAGATCAAGTTTCAGGTGCACGTCGGTGAACC','AATCCACCAGCTCCACGTGCAATGTTGGCCTA']

gibbs_sampler(dna,8,5,100)

['TCTCGGGG', 'CCAAGGTG', 'TACAGGCG', 'TTCAGGTG', 'TCCACGTG']